![LVMM logo](https://lvmm.mx/wp-content/uploads/2019/05/LVMM_logo_500-90x90.png)

# Manuales de uso del clúster LVMM

<a name="ssh"></a>
## 1. ¿Cómo acceder al cluster LVMM?

La coneción al clúster computacional del Departamento de Modelación de Nanomateriales es por medio del protocolo [OpenSSH](https://www.openssh.com/) (Open Secure Shell, por sus siglas en inglés), dentro de la red del CNyN ésto se realiza por medio del siguiente comando:

```shell
$ ssh usuario@192.168.100.237
```

donde `usuario` se refiere al nombre del usuario al que se quiere acceder en el clúster, con éste comando nos preguntará nuestra clave:

```shell
By accessing this system, you consent to the following conditions:
- This system is for authorized use only.
- Any or all uses of this system and all files on this system may be monitored.
- Communications using, or data stored on, this system are not private.

usuario@192.168.100.237's password:

```

<a name="module"></a>
# 2. ¿Cómo utilizar los programas instalados en el clúster LVMM?

<a name="moduleav"></a>
Para poder utilizar los programas ya instalabos en el cúster LVMM es necesario cargar el módulo adecuado, ([Environment Modules](https://modules.sourceforge.net/)), la lista de aplicaciones, librerías y utilidades, se pueden consultar con el siguiente comando:

```shell
$ module available
```

o de forma adbreviada:

```shell
$ module av
```

```----------------------------------- /share/modules/Applications ------------------------------------
abinit/10.0.7.1-intel    gromacs/2024.1-gnu-mkl     vasp/6.4.1-intel           
amber/20-Update12        irrep/1.1                  vasp/6.4.1-intel-vtst         
autodock/4.2.6-intel     lammps/2Aug2023-intel      vasp/6.4.1-MD-intel           
boltztrap/20.7.1         namd/2.14-intel            vasp/6.4.1-nvhpc-gpu            
cp2k/2024.1-gnu-mkl      py4vasp/0.5.1              wannier90/2.1.0-intel
critic2/1.1.git-intel    qe/7.1-intel               wannier90/3.1.0-intel         
crystal17/1.0.2-intel    siesta/honpas-4.1.5-intel  wannier90/3.1.0-serial-intel  
dftb+/24.1-intel         tinker/8.10.2-intel        wien2k/23.2-intel             
gromacs/2024.1-gnu-cuda  

Key:
default-version  modulepath  
```

Si queremos utilizar, por ejemplo, el programa VASP para GPU, debemos primero cargar el módulo `vasp/6.4.1-nvhpc-gpu` con el siguiente comando:

```{coda-shell}{shell}
$ module load vasp/6.4.1-nvhpc-gpu
```


```
Loading vasp version 6.4.1
Loading mkl version 2023.0.0
Loading tbb version 2021.8.0
Loading compiler-rt version 2023.0.0
Loading fftw version 3.3.10
Loading hdf5 version 1.14.3

Loading vasp/6.4.1-nvhpc-gpu
  Loading requirement: nvhpc/24.5 tbb/latest compiler-rt/latest mkl/latest fftw/3.3.10-nvhpc
    hdf5/1.14.3-nvhpc
```

Los mensajes de salida indican que también se han cargado las dependencias requeridas para que funcione correctamente nuestra aplicación, una vez realizado ésto podémos correr `vasp_std`, `vasp_ncl` o `vasp_gam`.

```
$ mpirun -n 4 vasp_ncl
 running    4 mpi-ranks, with    2 threads/rank, on    1 nodes
 distrk:  each k-point on    4 cores,    1 groups
 distr:  one band on    1 cores,    4 groups
 vasp.6.4.1 05Apr23 (build Jul  1 2024 01:14:50) complex                         
 ```

con éste comando se ejecuta el programa `vasp_ncl` en el **nodo maestro**, lo que **NO es recomendado**).

<a name="slurm"></a>
# 3. ¿Cómo correr una aplicación en el LVMM?

El clúster HPC LVMM cuenta con el sistema de administración de recursos [SLURM](https://slurm.schedmd.com) (Simple Linux Utility for Resource Management, por sus siglas en inglés), con el comando `sinfo` podemos obtener información sobre el cluster,

<a name="sinfo"></a>
## 3.1 sinfo
```shell
$ sinfo
```
```shell
PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
gpu          up   infinite      2   idle compute-0-[0,5]
mem          up   infinite      1    mix compute-0-1
cpu*         up   infinite      3    mix compute-0-[1-2,4]
cpu*         up   infinite      1  alloc compute-0-3
```
éste comando nos muestra las **particiones** (colar) de ejecución de trabajos, podemos ver que ésta instalación de `SLURM` cuenta con 3 particiones distintas: `gpu`, `mem` y `cpu`, también podemos ver el éstado de los nodos de éstas particiones; La partición `gpu` se encuentra **desocupada** y que cuenta de los nodos de cómputo: `compute-0-0` y `compute-0-5`; también podemos ver que de la partición `cpu`, el nodo `compute-0-3` se encuentra **ocupado** y los nodos `compute-0-1`, `compute-0-2` y el nodo `compute-0-4` se encuentran en un éstado de ocupación **mixto**.

Podemos obtener la misma información pero ahora centrada en los nodos en lugar de las particiones con la opción `--Node` o en la forma abreviada `--N`,

```shell
$ sinfo --Node
NODELIST     NODES PARTITION STATE
compute-0-0      1       gpu idle  
compute-0-1      1      cpu* mix   
compute-0-1      1       mem mix   
compute-0-2      1      cpu* mix   
compute-0-3      1      cpu* alloc
compute-0-4      1      cpu* mix   
compute-0-5      1       gpu idle  
```

<a name="srun"></a>
## 3.2 srun

Para correr `vasp` en el cluster se puede ejecutar con la instrucción `srun` de la siguiente manera:

```shell
$ srun --partition mem --ntasks 16 vasp_ncl
```
```shell
 running   16 mpi-ranks, on    1 nodes
 distrk:  each k-point on   16 cores,    1 groups
 distr:  one band on    4 cores,    4 groups
 vasp.6.4.1 05Apr23 (build Feb 23 2024 02:44:29) complex                        
```

con ésto corremos el programa `vasp_ncl` en la partición `mem` con 15 tarea, el inconveniente de ésta forma de correr el programa es que tenemos que esperar hasta que estén disponibles los recursos requeridos y durante su ejecución.

<a name="sbatch"></a>
## 3.3 sbatch

La forma idonea para mandar un trabajo a una cola de ejecución es por medio del comando `sbatch`, para ésto tenemos que gerenar un script:

`vasp.job:`
```shell
#!/bin/bash
srun vasp_ncl
```
una vez creado el script se somete el script con el comando `sbatch`de la sigueinte forma:

```shell
$ sbatch --ntasks=16 --nodes=1 --partition=cpu vasp.job
```
```
Submitted batch job 176
```
ésto nos regresa un número `176` el cual es el `ID` del trabajo.

podemos agregar las opciones de la linea de comando de `sbatch` al mismo script, junto con otras opciones, de la siguiente forma:

`vasp.job:`
```shell
#!/bin/bash
#SBATCH --job-name=VASP_test        # Nombre para identificar el trabajo
#SBATCH --nodes=1                   # utilizar cólo un nodo
#SBATCH --ntasks=16                 # ejecitar 16 tareas
#SBATCH --cpus-per-task=1           # CPUs por tarea
#SBATCH --ntasks-per-node=16        # tareas por nodo
#SBATCH --mem-per-cpu=2048          # cantidad máxima de memoria alojada por cpu en MB
#SBATCH --output vasp_test-%j.o     # el nombre del archivo a donde escribir las salidas de la ejecución
#SBATCH --error  vasp_test-%j.e     # nombre del archivo para escribir los errores de ejecución
# %j se refiere al ID asignado a la hora de someter el script
#SBATCH --partition=cpu             # partición en la cual ejecutar el script


module load vasp/6.4.1-intel        # cargar el módulo adecuado

srun vasp_ncl                      # ejecutar vast_ncl en los recursos solicitados
```
para someter éste último script sólo tenemos que mandar como parametro el nombre del script (`vasp.job`) a `sbatch`:
```shell
$ sbatch vasp.job
```
```shell
Submitted batch job 177
```

<a name="squeue"></a>
## 3.4 squeue

Para ver el éstado de ejecución de los trabajos en la cola podemos utilizar la instrucción `squeue`.

```shell
$ squeue
```
```shell
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             164         cpu  model_4   jperez  R 16-10:19:13     1 compute-0-3
             174         cpu    model   jperez  R 4-07:57:35      1 compute-0-2
             173         cpu     VASP vgalingo  R 2-12:45:11      1 compute-0-1
             175         cpu     Test  wmedina  R   22:59:14      1 compute-0-2
             171         cpu      aex  wmedina  R 1-20:57:02      1 compute-0-3
             172         cpu   10-35s   clfdez  R    7:05:41      1 compute-0-1
             177         cpu VASP_tes    suser  R      16:32      1 compute-0-1
             178         cpu model_dm  emarmol  R    6:06:50      1 compute-0-3
             182         cpu model_fm vgalindo  R    5:47:16      1 compute-0-4
             184         cpu      igc  ncampos  R      56:32      1 compute-0-1
             186         mem Na2SO2_d    jvzmg  R    7:05:41      1 compute-0-1
```
podemor verificar que nuestro trabajo esta corriendo (R), podemos ver sólo nuestros trabajos con la opción `--user $USER`

```shell
$ squeue --user $USER
```
```shell
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             177         cpu VASP_tes    suser  R      18:22      1 compute-0-1
```

<a name="scancel"></a>
## 3.5 scancel
se puede cancelar el trabajo con el comando `scancel`:
```shell
$ scancel 177
```
``` shell
$ squeue -u $USER
```
```shell
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             177         cpu VASP_tes    suser  CA      18:22      1 compute-0-1
```
podemos ver que el trabajo cambió de estado a cancelado (CA). También podemos utilizar el nombre el trabajo `VASP_test` con la opción `--name`,
```shell
$ scancel --name VASP_test
```
<a name="salloc"></a>
## 3.6 salloc

También podemos ejecutar nuestros programas de una forma un poco más interactiva utilizando el comando `salloc`, el cual nos aloja los recursos solicitados y nos da un shell ineractivo
```shell
$ salloc -p cpu -N 1 -n 16 -J VASP_test -t 60
```
```shell
salloc: Granted job allocation 178
$
```
una vez terminada nuestra ejecución o pruebas nos podemos salir como en cualquier shell,
```shell
$ exit
```
```shell
salloc: Relinquishing job allocation 178
```


---
# 4. Referencias:

The SLURM website: https://slurm.schedmd.com/

The SLURM documentation: https://slurm.schedmd.com/documentation.html

The SLURM user community: https://groups.google.com/g/slurm-users

Aldo Rodríguez Guerrero, 2024